In [86]:
import pandas as pd

faktor_cest = pd.read_csv('./baze/manjse/FaktorCest.csv')
nesreca = pd.read_csv('./baze/manjse/Nesreca.csv')


merged_df = pd.merge(faktor_cest, nesreca[['ZaporednaStevilkaOsebeVPN', 'TipNesrece']], on='ZaporednaStevilkaOsebeVPN')

final_df = merged_df[['StanjeVozisca', 'TipNesrece']]

# odstrani vozisca in tipe nesrec v neznanem stanju
final_df = final_df.loc[final_df['StanjeVozisca'] != 'OSTALO']
final_df = final_df.loc[final_df['TipNesrece'] != 'OSTALO']

# print("Unique values in StanjeVozisca:", final_df['StanjeVozisca'].unique())
# print("\nUnique values in TipNesrece:", final_df['TipNesrece'].unique())


new_df = final_df.copy()

# obdrži vrstice s StanjeVozisca = SUHO, ostale pa preimenuj z DRUGACNO
new_df.loc[new_df['StanjeVozisca'] != 'SUHO', 'StanjeVozisca'] = 'DRUGACNO'
# print("Unique values in StanjeVozisca:", new_df['StanjeVozisca'].unique())


new_df['TipNesrece_new'] = 'SE PREMIKA'

mask = (new_df['TipNesrece'] == 'TRČENJE V STOJEČE / PARKIRANO VOZILO') | (new_df['TipNesrece'] == 'TRČENJE V OBJEKT')
new_df.loc[mask, 'TipNesrece_new'] = 'STACIONAREN'

mask = (new_df['TipNesrece'] == 'BOČNO TRČENJE') | (new_df['TipNesrece'] == 'OPLAŽENJE') | (new_df['TipNesrece'] == 'NALETNO TRČENJE') | (new_df['TipNesrece'] == 'POVOŽENJE ŽIVALI') | (new_df['TipNesrece'] == 'POVOŽENJE PEŠCA') | (new_df['TipNesrece'] == 'ČELNO TRČENJE')
new_df.loc[mask, 'TipNesrece_new'] = 'SE PREMIKA'

new_df.drop(columns=['TipNesrece'], inplace=True)
new_df.rename(columns={'TipNesrece_new': 'TipNesrece'}, inplace=True)

# print("Unique values in TipNesrece:", new_df['TipNesrece'].unique())

suhoSVC = new_df['StanjeVozisca'].value_counts()['SUHO']
drugacnoSVC = new_df['StanjeVozisca'].value_counts()['DRUGACNO']
sepremikaTNC = new_df['TipNesrece'].value_counts()['SE PREMIKA']
stacionarenTNC = new_df['TipNesrece'].value_counts()['STACIONAREN']

print("STOJI:", stacionarenTNC)
# print("PREMIKA:", sepremikaTNC)
print("SUHA:", suhoSVC)
print("MOKRA:", drugacnoSVC)


ms = len(new_df[(new_df['StanjeVozisca'] == 'DRUGACNO') & (new_df['TipNesrece'] == 'STACIONAREN')])
ss = len(new_df[(new_df['StanjeVozisca'] == 'SUHO') & (new_df['TipNesrece'] == 'STACIONAREN')])
# mp = len(new_df[(new_df['StanjeVozisca'] == 'DRUGACNO') & (new_df['TipNesrece'] == 'SE PREMIKA')])
# sp = len(new_df[(new_df['StanjeVozisca'] == 'SUHO') & (new_df['TipNesrece'] == 'SE PREMIKA')])

print("\nSTOJI & SUHA:", ss, "\nSTOJI & MOKRA:", ms)

r1 = suhoSVC / ss
r2 = drugacnoSVC / ms

# print("\nRazmerje stoji & suha:", r1)
# print("Razmerje stoji & mokra:", r2)

percentage_difference = abs((r1 - r2) / ((r1 + r2) / 2)) * 100

print(f"\nKarambolov v stacionarne objekte (npr. parkiran avto, stavba, itd.) na mokri cesti je bilo {round(percentage_difference, 4)}% več kot karambolov v stacionarne objekte na suhi cesti.")

STOJI: 7158
SUHA: 22534
MOKRA: 6749

STOJI & SUHA: 5444 
STOJI & MOKRA: 1714

Karambolov v stacionarne objekte (npr. parkiran avto, stavba, itd.) na mokri cesti je bilo 4.9936% več kot karambolov v stacionarne objekte na suhi cesti.
